In [35]:
# Imports
import os
from dotenv import load_dotenv
from pulsefire.clients import RiotAPIClient
from time import sleep
import sqlite3
from tqdm import tqdm


# Read .env and get RIOT API KEY
load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")

# SQLite
con = sqlite3.connect("league.db")
cur = con.cursor()


In [36]:
async def get_account_data(client, entries, league="challenger"):
  """
    Sleep commands are there to avoid hitting the rate limit
    Current rate limit: 100 requests every 2 minutes
    Hence the sleep(1.5) -> ~80 requests every 2 minutes
  """
  entry_id = 0 # running index
  accounts = [] # list of accounts
  pbar = tqdm(total=len(entries["entries"]), desc="Getting account data")
  while entry_id < len(entries["entries"]):
    try:
      entry = entries["entries"][entry_id]
      sleep(1.5)
      account = await client.get_account_v1_by_puuid(region="europe", puuid=entry['puuid'])
      
      # Create account dictionary
      account_data = {
        'player_id': entry['puuid'],
        'summoner_id': entry['summonerId'],
        'game_name': account['gameName'],
        'tag_line': account['tagLine'],
        'league': league,
        'rank': entry_id + 1,
        'LP': entry['leaguePoints'],
        'wins': entry['wins'],
        'losses': entry['losses'],
      }
      accounts.append(account_data)
      entry_id += 1
      pbar.update(1)
    except Exception as e:
      print("--------------ERROR--------------")
      print("Error in the loop:")
      print(e)
      print("--------------ERROR--------------")
      sleep(10)

  return accounts  

In [37]:
def insert_into_leaderboards(accounts, cur, con):
  for account in accounts:
    cur.execute("INSERT INTO leaderboards(player_id, summoner_id, game_name, tag_line, league, rank, league_points, wins, losses) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (account['player_id'], account['summoner_id'], account['game_name'], account['tag_line'], account['league'], account['rank'], account['LP'], account['wins'], account['losses']))
  con.commit()

In [38]:
# Challengers (300 people)
async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
  try:
    challengers = await client.get_lol_league_v4_challenger_league_by_queue(region="euw1", queue="RANKED_SOLO_5x5")
    print(f"Num of challenger players: {len(challengers['entries'])}")
    print("---------------------------------")
  except Exception as e:
    print("Cannot get league data")
    print(e)
    
  c_accounts = await get_account_data(client, challengers, league="challenger")


Num of challenger players: 300
---------------------------------


Getting account data: 100%|██████████| 300/300 [08:04<00:00,  1.62s/it]


In [40]:
# Grandmasters (700 people)
async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
  try:
    grandmasters = await client.get_lol_league_v4_grandmaster_league_by_queue(region="euw1", queue="RANKED_SOLO_5x5")
    print(f"Num of grandmaster players: {len(grandmasters['entries'])}")
    print("---------------------------------")
  except Exception as e:
    print("Cannot get league data")
    print(e)
  
  gm_accounts = await get_account_data(client, grandmasters, league="grandmaster")

Num of grandmaster players: 700
---------------------------------


Getting account data: 100%|██████████| 700/700 [18:57<00:00,  1.63s/it]


In [39]:
# Insert challengers into leaderboards
insert_into_leaderboards(c_accounts, cur, con)

In [41]:
# Insert grandmasters into leaderboards
insert_into_leaderboards(gm_accounts, cur, con)


In [42]:
con.close()